In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import datetime

from tqdm import tqdm

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

feat_no = '109_ker_'

### Data Load

In [113]:
def elo_load_data(filename=''):
    
    if len(filename):
        df = utils.read_df_pkl(path=f'../input/{filename}*.p')
        return df
    # read pickle
    path_list = glob.glob("../input/*.p")
    for path in path_list:
        filename = re.search(r'/([^/.]*).gz', path).group(1)
        df = utils.read_df_pkl(path=f'../input/{filename}*.p')
        
train = elo_load_data('train')
test = elo_load_data('test')
train['first_active_month'] = pd.to_datetime(train['first_active_month'])
test['first_active_month'] = pd.to_datetime(test['first_active_month'])
train['elapsed_days'] = (datetime.date(2018, 2, 1) - train['first_active_month'].dt.date).dt.days
test['elapsed_days'] = (datetime.date(2018, 2, 1) - test['first_active_month'].dt.date).dt.days
train_id = train[key].values
test_id = test[key].values
new = elo_load_data('new_')
historical = elo_load_data('historical')

100%|██████████| 3/3 [00:00<00:00, 176.11it/s]


SystemExit: 

In [3]:
def binarize(df):
    for col in ['authorized_flag', 'category_1']:
        df[col] = df[col].map({'Y':1, 'N':0})
    return df

historical = binarize(historical)
new = binarize(new)

In [114]:
y = train[target]
del train[target]
train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,elapsed_days
0,2017-06-01,C_ID_92a2005557,5,2,1,245
1,2017-01-01,C_ID_3d0044924f,4,1,0,396
2,2016-08-01,C_ID_d639edf6cd,2,2,0,549
3,2017-09-01,C_ID_186d6a6901,4,3,0,153
4,2017-11-01,C_ID_cdbd2c0db2,1,3,0,92


In [5]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [6]:
historical = pd.get_dummies(historical, columns=['category_2', 'category_3'])
new = pd.get_dummies(new, columns=['category_2', 'category_3'])

historical = reduce_mem_usage(historical)
new = reduce_mem_usage(new)

agg_func = {'authorized_flag': ['sum', 'mean']}
auth_mean = historical.groupby(key).agg(agg_func)
auth_mean.head()

Mem. usage decreased to 1527.00 Mb (50.9% reduction)
Mem. usage decreased to 102.97 Mb (51.8% reduction)


authorized_flag          
                            sum      mean
card_id                                  
C_ID_00007093c1           114.0  0.765101
C_ID_0001238066           120.0  0.975610
C_ID_0001506ef0            62.0  0.939394
C_ID_0001793786           189.0  0.875000
C_ID_000183fdda           137.0  0.951389

In [32]:
auth_mean.columns = ['_'.join(col).strip() for col in auth_mean.columns.values]
auth_mean.reset_index(inplace=True)
display(auth_mean.head())

auth_1 = historical[historical['authorized_flag']==1]
auth_0 = historical[historical['authorized_flag']==0]

In [33]:
auth_0[0:5]

,authorized_flag,card_id,city_id,category_1,installments,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,state_id,subsector_id,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,category_3_A,category_3_B,category_3_C
115,0,C_ID_4e6213e9bc,88,0,0,842,M_ID_22c9cfa265,-10,-0.730379,2017-04-07 12:58:09,16,37,1,0,0,0,0,1,0,0
132,0,C_ID_4e6213e9bc,88,0,0,367,M_ID_86ec983688,-5,-0.723782,2017-09-17 22:40:27,16,16,1,0,0,0,0,1,0,0
148,0,C_ID_4e6213e9bc,88,0,0,367,M_ID_86ec983688,-5,-0.723782,2017-09-17 22:40:26,16,16,1,0,0,0,0,1,0,0
168,0,C_ID_4e6213e9bc,333,0,0,605,M_ID_c2ae34c2ef,0,-0.664262,2018-02-20 10:57:50,9,2,1,0,0,0,0,1,0,0
213,0,C_ID_4e6213e9bc,88,0,0,560,M_ID_e6d5ae8ea6,-7,-0.738132,2017-07-08 07:33:31,16,34,1,0,0,0,0,1,0,0


### Feature Engineering

In [34]:
auth_1['purchase_date'] = pd.to_datetime(auth_1['purchase_date'])
auth_0['purchase_date'] = pd.to_datetime(auth_0['purchase_date'])
auth_1['purchase_month'] = auth_1['purchase_date'].dt.month
auth_0['purchase_month'] = auth_0['purchase_date'].dt.month
new['purchase_month'] = pd.to_datetime(new['purchase_date']).dt.month

In [69]:
def aggregate_transactions(history):
    
    history.loc[:, 'purchase_date'] = pd.DatetimeIndex(history['purchase_date'].astype(np.int64)*1e-9)
    
    agg_func = {
        'category_1' : ['sum', 'mean'],
        'category_2_1.0' : ['sum', 'mean'],
        'category_2_2.0' : ['sum', 'mean'],
        'category_2_3.0' : ['sum', 'mean'],
        'category_2_4.0' : ['sum', 'mean'],
        'category_2_5.0' : ['sum', 'mean'],
        'category_3_A' : ['sum', 'mean'],
        'category_3_B' : ['sum', 'mean'],
        'category_3_C' : ['sum', 'mean'],
        'merchant_id' : ['nunique'],
        'state_id' : ['nunique'],
        'city_id' : ['nunique'],
        'subsector_id' : ['nunique'],
        'purchase_amount' : ['sum', 'mean', 'max', 'min', 'std', 'skew', np.ptp],
        'installments' : ['sum', 'mean', 'max', 'min', 'std', 'skew', np.ptp],
        'purchase_month' : ['mean', 'max', 'min', 'std', 'skew', np.ptp],
        'purchase_date' : [np.ptp],
        'month_lag' : ['min', 'max', np.ptp]
    }
    
    agg_history = history.groupby(['card_id']).agg(agg_func)
    agg_history.columns = ['_'.join(col).strip() for col in agg_history.columns.values]
    agg_history.reset_index(inplace=True)
    
    df = history.groupby(key).size().reset_index(name='transactions_cnt')
    
    agg_history = pd.merge(df, agg_history, on=key, how='left')
    
    return agg_history

In [70]:
auth_0_agg = aggregate_transactions(auth_0)
auth_0_agg.columns = ['auth0_' + col + '@' if col!=key else col for col in auth_0_agg.columns]
auth_0_agg[:5]

,card_id,auth0_transactions_cnt@,auth0_category_1_sum@,auth0_category_1_mean@,auth0_category_2_1.0_sum@,auth0_category_2_1.0_mean@,auth0_category_2_2.0_sum@,auth0_category_2_2.0_mean@,auth0_category_2_3.0_sum@,auth0_category_2_3.0_mean@,auth0_category_2_4.0_sum@,auth0_category_2_4.0_mean@,auth0_category_2_5.0_sum@,auth0_category_2_5.0_mean@,auth0_category_3_A_sum@,auth0_category_3_A_mean@,auth0_category_3_B_sum@,auth0_category_3_B_mean@,auth0_category_3_C_sum@,auth0_category_3_C_mean@,auth0_merchant_id_nunique@,auth0_state_id_nunique@,auth0_city_id_nunique@,auth0_subsector_id_nunique@,auth0_purchase_amount_sum@,auth0_purchase_amount_mean@,auth0_purchase_amount_max@,auth0_purchase_amount_min@,auth0_purchase_amount_std@,auth0_purchase_amount_skew@,auth0_purchase_amount_ptp@,auth0_installments_sum@,auth0_installments_mean@,auth0_installments_max@,auth0_installments_min@,auth0_installments_std@,auth0_installments_skew@,auth0_installments_ptp@,auth0_purchase_month_mean@,auth0_purchase_month_max@,auth0_purchase_month_min@,auth0_purchase_month_std@,auth0_purchase_month_skew@,auth0_purchase_month_ptp@,auth0_purchase_date_ptp@,auth0_month_lag_min@,auth0_month_lag_max@,auth0_month_lag_ptp@
0,C_ID_00007093c1,35,4.0,0.114286,0.0,0.000000,0,0.000000,31,0.885714,0.0,0.0,0.0,0.000000,0.0,0.000000,29.0,0.828571,6.0,0.171429,11,2,3,7,-14.401965,-0.411485,1.507069,-0.728876,0.430059,3.035876,2.235945,45,1.285714,3,1,0.667367,2.120415,2,5.914286,12,1,3.071419,0.195403,11,0 days,-11,0,11
1,C_ID_0001238066,3,0.0,0.000000,1.0,0.333333,0,0.000000,0,0.000000,0.0,0.0,2.0,0.666667,0.0,0.000000,3.0,1.000000,0.0,0.000000,2,2,2,1,-1.789928,-0.596643,-0.596643,-0.596643,0.000000,0.000000,0.000000,3,1.000000,1,1,0.000000,0.000000,0,8.666667,12,2,5.773503,-1.732051,10,0 days,-2,0,2
2,C_ID_0001506ef0,4,0.0,0.000000,0.0,0.000000,0,0.000000,4,1.000000,0.0,0.0,0.0,0.000000,3.0,0.750000,1.0,0.250000,0.0,0.000000,4,1,1,4,-2.905782,-0.726445,-0.708590,-0.740491,0.016203,0.218985,0.031901,1,0.250000,1,0,0.500000,2.000000,1,5.500000,12,2,4.725816,1.193824,10,0 days,-8,0,8
3,C_ID_0001793786,27,2.0,0.074074,3.0,0.111111,8,0.296296,3,0.111111,0.0,0.0,0.0,0.000000,22.0,0.814815,5.0,0.185185,0.0,0.000000,17,4,8,11,-11.825338,-0.437975,1.236592,-0.745405,0.498150,3.104427,1.981997,5,0.185185,1,0,0.395847,1.717834,1,6.481481,9,3,2.375684,-0.402658,6,0 days,-7,-1,6
4,C_ID_000183fdda,7,0.0,0.000000,0.0,0.000000,0,0.000000,7,1.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,3.0,0.428571,4.0,0.571429,4,1,1,4,-0.941381,-0.134483,0.334099,-0.714541,0.452741,0.113341,1.048640,19,2.714286,6,1,2.288689,1.072468,5,8.285714,10,8,0.755929,2.645751,2,0 days,-6,-4,2


In [71]:
auth_1_agg = aggregate_transactions(auth_1)
auth_1_agg.columns = ['auth1_' + col + '@' if col!=key else col for col in auth_1_agg.columns]
auth_1_agg[:5]

,card_id,auth1_transactions_cnt@,auth1_category_1_sum@,auth1_category_1_mean@,auth1_category_2_1.0_sum@,auth1_category_2_1.0_mean@,auth1_category_2_2.0_sum@,auth1_category_2_2.0_mean@,auth1_category_2_3.0_sum@,auth1_category_2_3.0_mean@,auth1_category_2_4.0_sum@,auth1_category_2_4.0_mean@,auth1_category_2_5.0_sum@,auth1_category_2_5.0_mean@,auth1_category_3_A_sum@,auth1_category_3_A_mean@,auth1_category_3_B_sum@,auth1_category_3_B_mean@,auth1_category_3_C_sum@,auth1_category_3_C_mean@,auth1_merchant_id_nunique@,auth1_state_id_nunique@,auth1_city_id_nunique@,auth1_subsector_id_nunique@,auth1_purchase_amount_sum@,auth1_purchase_amount_mean@,auth1_purchase_amount_max@,auth1_purchase_amount_min@,auth1_purchase_amount_std@,auth1_purchase_amount_skew@,auth1_purchase_amount_ptp@,auth1_installments_sum@,auth1_installments_mean@,auth1_installments_max@,auth1_installments_min@,auth1_installments_std@,auth1_installments_skew@,auth1_installments_ptp@,auth1_purchase_month_mean@,auth1_purchase_month_max@,auth1_purchase_month_min@,auth1_purchase_month_std@,auth1_purchase_month_skew@,auth1_purchase_month_ptp@,auth1_purchase_date_ptp@,auth1_month_lag_min@,auth1_month_lag_max@,auth1_month_lag_ptp@
0,C_ID_00007093c1,114,24.0,0.210526,0.0,0.000000,0.0,0.000000,89.0,0.780702,0.0,0.0,1.0,0.008772,0.0,0.0,96.0,0.842105,18.0,0.157895,28,3,4,13,-62.443077,-0.547746,1.206539,-0.728876,0.237426,4.183977,1.935415,147,1.289474,6,1,0.795159,3.514956,5,6.517544,12,1,3.371490,-0.010131,11,0 days,-12,0,12
1,C_ID_0001238066,120,2.0,0.016667,94.0,0.783333,0.0,0.000000,0.0,0.000000,0.0,0.0,18.0,0.150000,0.0,0.0,85.0,0.708333,32.0,0.266667,65,6,18,17,-70.657272,-0.588811,0.768095,-0.734887,0.192614,4.006112,1.502982,195,1.625000,10,-1,1.501050,2.893080,11,7.275000,12,1,4.895483,-0.341349,11,0 days,-5,0,5
2,C_ID_0001506ef0,62,0.0,0.000000,2.0,0.032258,0.0,0.000000,60.0,0.967742,0.0,0.0,0.0,0.000000,62.0,1.0,0.0,0.000000,0.0,0.000000,28,2,3,12,-31.696098,-0.511227,1.493545,-0.740491,0.484575,2.583079,2.234036,0,0.000000,0,0,0.000000,0.000000,0,6.887097,12,1,4.538017,-0.089439,11,0 days,-13,0,13
3,C_ID_0001793786,189,0.0,0.000000,8.0,0.042328,68.0,0.359788,12.0,0.063492,0.0,0.0,0.0,0.000000,189.0,1.0,0.0,0.000000,0.0,0.000000,114,4,9,22,-24.960674,-0.132067,4.554145,-0.737892,0.867916,2.909714,5.292037,0,0.000000,0,0,0.000000,0.000000,0,6.698413,10,1,2.301491,-0.414027,9,0 days,-9,0,9
4,C_ID_000183fdda,137,4.0,0.029197,7.0,0.051095,1.0,0.007299,124.0,0.905109,0.0,0.0,1.0,0.007299,0.0,0.0,100.0,0.729927,33.0,0.240876,71,7,9,20,-67.896553,-0.495595,2.764788,-0.737892,0.523760,4.088245,3.502680,245,1.788321,10,-1,2.108912,2.799102,11,6.810219,12,1,4.538289,-0.263625,11,0 days,-5,0,5


In [72]:
new['purchase_date'] = pd.to_datetime(new['purchase_date'])
new_agg = aggregate_transactions(new)
new_agg.columns = ['new_' + col + '@' if col!=key else col for col in new_agg.columns]
new_agg[:5]

,card_id,new_transactions_cnt@,new_category_1_sum@,new_category_1_mean@,new_category_2_1.0_sum@,new_category_2_1.0_mean@,new_category_2_2.0_sum@,new_category_2_2.0_mean@,new_category_2_3.0_sum@,new_category_2_3.0_mean@,new_category_2_4.0_sum@,new_category_2_4.0_mean@,new_category_2_5.0_sum@,new_category_2_5.0_mean@,new_category_3_A_sum@,new_category_3_A_mean@,new_category_3_B_sum@,new_category_3_B_mean@,new_category_3_C_sum@,new_category_3_C_mean@,new_merchant_id_nunique@,new_state_id_nunique@,new_city_id_nunique@,new_subsector_id_nunique@,new_purchase_amount_sum@,new_purchase_amount_mean@,new_purchase_amount_max@,new_purchase_amount_min@,new_purchase_amount_std@,new_purchase_amount_skew@,new_purchase_amount_ptp@,new_installments_sum@,new_installments_mean@,new_installments_max@,new_installments_min@,new_installments_std@,new_installments_skew@,new_installments_ptp@,new_purchase_month_mean@,new_purchase_month_max@,new_purchase_month_min@,new_purchase_month_std@,new_purchase_month_skew@,new_purchase_month_ptp@,new_purchase_date_ptp@,new_month_lag_min@,new_month_lag_max@,new_month_lag_ptp@
0,C_ID_00007093c1,2,0,0.000000,1,0.500000,0,0.000000,1,0.50000,0,0.0,0,0.000000,0,0.0,2,1.000000,0,0.000000,2,2,2,2,-1.328125,-0.664062,-0.656738,-0.671875,0.010704,NaN,0.015137,2.0,1.000000,1.0,1.0,0.000000,NaN,0.0,4.000000,4,4,0.000000,NaN,0,0 days,2,2,0
1,C_ID_0001238066,26,2,0.076923,20,0.769231,0,0.000000,0,0.00000,0,0.0,3,0.115385,0,0.0,21,0.807692,4,0.153846,26,4,8,9,-14.851562,-0.571289,-0.078308,-0.740723,0.173462,1.569336,0.662598,43.0,1.719727,10.0,1.0,2.052734,3.369141,9.0,3.346154,4,3,0.485165,0.687052,1,0 days,1,2,1
2,C_ID_0001506ef0,2,0,0.000000,0,0.000000,0,0.000000,2,1.00000,0,0.0,0,0.000000,2,1.0,0,0.000000,0,0.000000,2,1,1,2,-1.447266,-0.723633,-0.715332,-0.731934,0.011742,NaN,0.016602,0.0,0.000000,0.0,0.0,0.000000,NaN,0.0,3.000000,3,3,0.000000,NaN,0,0 days,1,1,0
3,C_ID_0001793786,31,0,0.000000,15,0.483871,8,0.258065,5,0.16129,0,0.0,1,0.032258,31,1.0,0,0.000000,0,0.000000,31,5,7,14,-0.229004,-0.007385,3.130859,-0.737793,0.947266,2.197266,3.869141,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,11.322581,12,11,0.475191,0.798227,1,0 days,1,2,1
4,C_ID_000183fdda,11,0,0.000000,0,0.000000,0,0.000000,11,1.00000,0,0.0,0,0.000000,0,0.0,6,0.545455,4,0.363636,11,2,2,6,-6.589844,-0.599121,-0.107666,-0.732422,0.182861,2.275391,0.625000,17.0,1.700195,4.0,1.0,1.059570,1.444336,3.0,3.272727,4,3,0.467099,1.189373,1,0 days,1,2,1


In [75]:
def aggregate_per_month(history):
    grouped = history.groupby([key, 'month_lag'])
    
    agg_func = {
        'purchase_amount':['count', 'sum', 'mean', 'min', 'max', 'std', np.ptp],
        'installments':['count', 'sum', 'mean', 'min', 'max', 'std', np.ptp],
    }
    
    # card_id month_lag level aggregation
    tmp_group = grouped.agg(agg_func)
    tmp_group.columns = ['_'.join(col).strip() for col in tmp_group.columns.values]
    tmp_group.reset_index(inplace=True)
    
    # card_id level aggregation
    result = tmp_group.groupby(key).agg(['mean', 'std'])
    
    return result

In [76]:
auth_1_agg_month = aggregate_per_month(auth_1)
auth_1_agg_month.columns = ['auth1_2level_' + '_'.join(col).strip() + '@' for col in auth_1_agg_month.columns.values]
auth_0_agg_month = aggregate_per_month(auth_0)
auth_0_agg_month.columns = ['auth0_2level_' + '_'.join(col).strip() + '@' for col in auth_0_agg_month.columns.values]
new_agg_month = aggregate_per_month(new)
new_agg_month.columns = ['new_2level_' + '_'.join(col).strip() + '@' for col in new_agg_month.columns.values]
display(auth_1_agg_month.head())
display(auth_0_agg_month.head())

,auth1_2level_month_lag_mean@,auth1_2level_month_lag_std@,auth1_2level_purchase_amount_count_mean@,auth1_2level_purchase_amount_count_std@,auth1_2level_purchase_amount_sum_mean@,auth1_2level_purchase_amount_sum_std@,auth1_2level_purchase_amount_mean_mean@,auth1_2level_purchase_amount_mean_std@,auth1_2level_purchase_amount_min_mean@,auth1_2level_purchase_amount_min_std@,auth1_2level_purchase_amount_max_mean@,auth1_2level_purchase_amount_max_std@,auth1_2level_purchase_amount_std_mean@,auth1_2level_purchase_amount_std_std@,auth1_2level_purchase_amount_ptp_mean@,auth1_2level_purchase_amount_ptp_std@,auth1_2level_installments_count_mean@,auth1_2level_installments_count_std@,auth1_2level_installments_sum_mean@,auth1_2level_installments_sum_std@,auth1_2level_installments_mean_mean@,auth1_2level_installments_mean_std@,auth1_2level_installments_min_mean@,auth1_2level_installments_min_std@,auth1_2level_installments_max_mean@,auth1_2level_installments_max_std@,auth1_2level_installments_std_mean@,auth1_2level_installments_std_std@,auth1_2level_installments_ptp_mean@,auth1_2level_installments_ptp_std@
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_00007093c1,-6.000000,3.894440,8.769231,3.539158,-4.803313,2.175011,-0.536131,0.075418,-0.702649,0.041293,-0.126683,0.446167,0.197339,0.120199,0.575966,0.448272,8.769231,3.539158,11.307692,4.210792,1.369646,0.423190,1.000000,0.000000,2.538462,1.613246,0.625428,0.675607,1.538462,1.613246
C_ID_0001238066,-2.500000,1.870829,20.000000,11.696153,-11.776212,6.676914,-0.603215,0.046045,-0.730351,0.002699,-0.160977,0.485869,0.151393,0.093832,0.569375,0.488304,20.000000,11.696153,32.500000,24.501020,1.652873,0.410539,0.666667,0.816497,4.666667,2.875181,1.256525,0.521292,4.000000,3.162278
C_ID_0001506ef0,-6.230769,4.225988,4.769231,4.342692,-2.438161,2.902502,-0.446660,0.271125,-0.725724,0.011212,0.161333,0.748701,0.496308,0.415117,0.887057,0.749652,4.769231,4.342692,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
C_ID_0001793786,-4.500000,3.027650,18.900000,12.160501,-2.496067,4.018810,-0.005687,0.271221,-0.646277,0.166230,2.190924,1.290775,0.880276,0.259048,2.837201,1.336454,18.900000,12.160501,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
C_ID_000183fdda,-2.500000,1.870829,22.833333,7.167054,-11.316092,4.800612,-0.489227,0.141015,-0.728405,0.009876,1.151797,1.062785,0.472149,0.315407,1.880202,1.059485,22.833333,7.167054,40.833333,14.048725,1.834943,0.362557,0.333333,1.032796,9.166667,2.041241,2.144389,0.665529,8.833333,2.562551


,auth0_2level_month_lag_mean@,auth0_2level_month_lag_std@,auth0_2level_purchase_amount_count_mean@,auth0_2level_purchase_amount_count_std@,auth0_2level_purchase_amount_sum_mean@,auth0_2level_purchase_amount_sum_std@,auth0_2level_purchase_amount_mean_mean@,auth0_2level_purchase_amount_mean_std@,auth0_2level_purchase_amount_min_mean@,auth0_2level_purchase_amount_min_std@,auth0_2level_purchase_amount_max_mean@,auth0_2level_purchase_amount_max_std@,auth0_2level_purchase_amount_std_mean@,auth0_2level_purchase_amount_std_std@,auth0_2level_purchase_amount_ptp_mean@,auth0_2level_purchase_amount_ptp_std@,auth0_2level_installments_count_mean@,auth0_2level_installments_count_std@,auth0_2level_installments_sum_mean@,auth0_2level_installments_sum_std@,auth0_2level_installments_mean_mean@,auth0_2level_installments_mean_std@,auth0_2level_installments_min_mean@,auth0_2level_installments_min_std@,auth0_2level_installments_max_mean@,auth0_2level_installments_max_std@,auth0_2level_installments_std_mean@,auth0_2level_installments_std_std@,auth0_2level_installments_ptp_mean@,auth0_2level_installments_ptp_std@
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_00007093c1,-5.500000,3.605551,2.916667,1.621354,-1.200164,0.999049,-0.431906,0.260478,-0.613764,0.156374,-0.215325,0.617379,0.273154,0.389389,0.398439,0.645234,2.916667,1.621354,3.750000,3.107908,1.176389,0.326634,1.000000,0.000000,1.500000,0.797724,0.355262,0.480551,0.500000,0.797724
C_ID_0001238066,-1.000000,1.414214,1.500000,0.707107,-0.894964,0.421890,-0.596643,0.000000,-0.596643,0.000000,-0.596643,0.000000,0.000000,NaN,0.000000,0.000000,1.500000,0.707107,1.500000,0.707107,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,NaN,0.000000,0.000000
C_ID_0001506ef0,-3.333333,4.163332,1.333333,0.577350,-0.968594,0.395645,-0.731020,0.015849,-0.732397,0.013464,-0.729642,0.018235,0.005844,NaN,0.002755,0.004772,1.333333,0.577350,0.333333,0.577350,0.166667,0.288675,0.000000,0.000000,0.333333,0.577350,0.707107,NaN,0.333333,0.577350
C_ID_0001793786,-3.833333,2.316607,4.500000,1.974842,-1.970890,2.037261,-0.423501,0.392159,-0.676499,0.054207,-0.139516,0.689409,0.278960,0.358014,0.536982,0.646841,4.500000,1.974842,0.833333,0.983192,0.291667,0.400520,0.166667,0.408248,0.500000,0.547723,0.179558,0.279244,0.333333,0.516398
C_ID_000183fdda,-5.000000,1.414214,3.500000,3.535534,-0.470690,0.344856,-0.376174,0.478523,-0.579708,0.190682,-0.190221,0.741501,0.409220,NaN,0.389487,0.550818,3.500000,3.535534,9.500000,12.020815,2.000000,1.414214,1.000000,0.000000,3.500000,3.535534,2.366432,NaN,2.500000,3.535534


In [118]:
base = utils.read_df_pkl('../input/base0*').set_index(key)
train_test = pd.concat([train, test], axis=0).set_index(key)
train_test.columns = [col+'@' if col not in ignore_list else col for col in train_test.columns if col not in ignore_list]
df_feature = base.join(train_test).join(auth_1_agg.set_index(key)).join(auth_0_agg.set_index(key)).join(new_agg.set_index(key)).join(auth_1_agg_month).join(auth_0_agg_month).join(new_agg_month)
df_feature.head()

SystemExit: 

In [130]:
df = df_feature
train_feat = df.loc[train_id, :]
test_feat = df.loc[test_id, :]
for col in df.columns:
    if col.count('@'):
        utils.to_pkl_gzip(obj=train_feat[col].values, path=f'../features/1_first_valid/{feat_no}train_{col}')
        utils.to_pkl_gzip(obj=test_feat[col].values, path=f'../features/1_first_valid/{feat_no}test_{col}')

In [123]:
date_cols = [col for col in df_feature.columns if str(type(df_feature[col].values[0])).count('time')]
for col in date_cols:
    df_feature[col] = df_feature[col].map(lambda x: x.days)
train = df_feature[~df_feature[target].isnull()]
test = df_feature[df_feature[target].isnull()]
features = [c for c in train.columns if c not in ['card_id', 'first_active_month', target]]
categorical_feats = [c for c in features if 'feature_' in c]

In [132]:
display(test_feat.head())
display(test.head())
display(test_feat.tail())
display(test.tail())

,target,feature_1@,feature_2@,feature_3@,elapsed_days@,auth1_transactions_cnt@,auth1_category_1_sum@,auth1_category_1_mean@,auth1_category_2_1.0_sum@,auth1_category_2_1.0_mean@,auth1_category_2_2.0_sum@,auth1_category_2_2.0_mean@,auth1_category_2_3.0_sum@,auth1_category_2_3.0_mean@,auth1_category_2_4.0_sum@,auth1_category_2_4.0_mean@,auth1_category_2_5.0_sum@,auth1_category_2_5.0_mean@,auth1_category_3_A_sum@,auth1_category_3_A_mean@,auth1_category_3_B_sum@,auth1_category_3_B_mean@,auth1_category_3_C_sum@,auth1_category_3_C_mean@,auth1_merchant_id_nunique@,auth1_state_id_nunique@,auth1_city_id_nunique@,auth1_subsector_id_nunique@,auth1_purchase_amount_sum@,auth1_purchase_amount_mean@,auth1_purchase_amount_max@,auth1_purchase_amount_min@,auth1_purchase_amount_std@,auth1_purchase_amount_skew@,auth1_purchase_amount_ptp@,auth1_installments_sum@,auth1_installments_mean@,auth1_installments_max@,auth1_installments_min@,auth1_installments_std@,auth1_installments_skew@,auth1_installments_ptp@,auth1_purchase_month_mean@,auth1_purchase_month_max@,auth1_purchase_month_min@,auth1_purchase_month_std@,auth1_purchase_month_skew@,auth1_purchase_month_ptp@,auth1_purchase_date_ptp@,auth1_month_lag_min@,auth1_month_lag_max@,auth1_month_lag_ptp@,auth0_transactions_cnt@,auth0_category_1_sum@,auth0_category_1_mean@,auth0_category_2_1.0_sum@,auth0_category_2_1.0_mean@,auth0_category_2_2.0_sum@,auth0_category_2_2.0_mean@,auth0_category_2_3.0_sum@,auth0_category_2_3.0_mean@,auth0_category_2_4.0_sum@,auth0_category_2_4.0_mean@,auth0_category_2_5.0_sum@,auth0_category_2_5.0_mean@,auth0_category_3_A_sum@,auth0_category_3_A_mean@,auth0_category_3_B_sum@,auth0_category_3_B_mean@,auth0_category_3_C_sum@,auth0_category_3_C_mean@,auth0_merchant_id_nunique@,auth0_state_id_nunique@,auth0_city_id_nunique@,auth0_subsector_id_nunique@,auth0_purchase_amount_sum@,auth0_purchase_amount_mean@,auth0_purchase_amount_max@,auth0_purchase_amount_min@,auth0_purchase_amount_std@,auth0_purchase_amount_skew@,auth0_purchase_amount_ptp@,auth0_installments_sum@,auth0_installments_mean@,auth0_installments_max@,auth0_installments_min@,auth0_installments_std@,auth0_installments_skew@,auth0_installments_ptp@,auth0_purchase_month_mean@,auth0_purchase_month_max@,auth0_purchase_month_min@,auth0_purchase_month_std@,auth0_purchase_month_skew@,auth0_purchase_month_ptp@,auth0_purchase_date_ptp@,auth0_month_lag_min@,auth0_month_lag_max@,auth0_month_lag_ptp@,new_transactions_cnt@,...,new_purchase_month_mean@,new_purchase_month_max@,new_purchase_month_min@,new_purchase_month_std@,new_purchase_month_skew@,new_purchase_month_ptp@,new_purchase_date_ptp@,new_month_lag_min@,new_month_lag_max@,new_month_lag_ptp@,auth1_2level_month_lag_mean@,auth1_2level_month_lag_std@,auth1_2level_purchase_amount_count_mean@,auth1_2level_purchase_amount_count_std@,auth1_2level_purchase_amount_sum_mean@,auth1_2level_purchase_amount_sum_std@,auth1_2level_purchase_amount_mean_mean@,auth1_2level_purchase_amount_mean_std@,auth1_2level_purchase_amount_min_mean@,auth1_2level_purchase_amount_min_std@,auth1_2level_purchase_amount_max_mean@,auth1_2level_purchase_amount_max_std@,auth1_2level_purchase_amount_std_mean@,auth1_2level_purchase_amount_std_std@,auth1_2level_purchase_amount_ptp_mean@,auth1_2level_purchase_amount_ptp_std@,auth1_2level_installments_count_mean@,auth1_2level_installments_count_std@,auth1_2level_installments_sum_mean@,auth1_2level_installments_sum_std@,auth1_2level_installments_mean_mean@,auth1_2level_installments_mean_std@,auth1_2level_installments_min_mean@,auth1_2level_installments_min_std@,auth1_2level_installments_max_mean@,auth1_2level_installments_max_std@,auth1_2level_installments_std_mean@,auth1_2level_installments_std_std@,auth1_2level_installments_ptp_mean@,auth1_2level_installments_ptp_std@,auth0_2level_month_lag_mean@,auth0_2level_month_lag_std@,auth0_2level_purchase_amount_count_mean@,auth0_2level_purchase_amount_count_std@,auth0_2level_purchase_amount_sum_mean@,auth0_2level_purchase_amount_su

,target,feature_1@,feature_2@,feature_3@,elapsed_days@,auth1_transactions_cnt@,auth1_category_1_sum@,auth1_category_1_mean@,auth1_category_2_1.0_sum@,auth1_category_2_1.0_mean@,auth1_category_2_2.0_sum@,auth1_category_2_2.0_mean@,auth1_category_2_3.0_sum@,auth1_category_2_3.0_mean@,auth1_category_2_4.0_sum@,auth1_category_2_4.0_mean@,auth1_category_2_5.0_sum@,auth1_category_2_5.0_mean@,auth1_category_3_A_sum@,auth1_category_3_A_mean@,auth1_category_3_B_sum@,auth1_category_3_B_mean@,auth1_category_3_C_sum@,auth1_category_3_C_mean@,auth1_merchant_id_nunique@,auth1_state_id_nunique@,auth1_city_id_nunique@,auth1_subsector_id_nunique@,auth1_purchase_amount_sum@,auth1_purchase_amount_mean@,auth1_purchase_amount_max@,auth1_purchase_amount_min@,auth1_purchase_amount_std@,auth1_purchase_amount_skew@,auth1_purchase_amount_ptp@,auth1_installments_sum@,auth1_installments_mean@,auth1_installments_max@,auth1_installments_min@,auth1_installments_std@,auth1_installments_skew@,auth1_installments_ptp@,auth1_purchase_month_mean@,auth1_purchase_month_max@,auth1_purchase_month_min@,auth1_purchase_month_std@,auth1_purchase_month_skew@,auth1_purchase_month_ptp@,auth1_purchase_date_ptp@,auth1_month_lag_min@,auth1_month_lag_max@,auth1_month_lag_ptp@,auth0_transactions_cnt@,auth0_category_1_sum@,auth0_category_1_mean@,auth0_category_2_1.0_sum@,auth0_category_2_1.0_mean@,auth0_category_2_2.0_sum@,auth0_category_2_2.0_mean@,auth0_category_2_3.0_sum@,auth0_category_2_3.0_mean@,auth0_category_2_4.0_sum@,auth0_category_2_4.0_mean@,auth0_category_2_5.0_sum@,auth0_category_2_5.0_mean@,auth0_category_3_A_sum@,auth0_category_3_A_mean@,auth0_category_3_B_sum@,auth0_category_3_B_mean@,auth0_category_3_C_sum@,auth0_category_3_C_mean@,auth0_merchant_id_nunique@,auth0_state_id_nunique@,auth0_city_id_nunique@,auth0_subsector_id_nunique@,auth0_purchase_amount_sum@,auth0_purchase_amount_mean@,auth0_purchase_amount_max@,auth0_purchase_amount_min@,auth0_purchase_amount_std@,auth0_purchase_amount_skew@,auth0_purchase_amount_ptp@,auth0_installments_sum@,auth0_installments_mean@,auth0_installments_max@,auth0_installments_min@,auth0_installments_std@,auth0_installments_skew@,auth0_installments_ptp@,auth0_purchase_month_mean@,auth0_purchase_month_max@,auth0_purchase_month_min@,auth0_purchase_month_std@,auth0_purchase_month_skew@,auth0_purchase_month_ptp@,auth0_purchase_date_ptp@,auth0_month_lag_min@,auth0_month_lag_max@,auth0_month_lag_ptp@,new_transactions_cnt@,...,new_purchase_month_mean@,new_purchase_month_max@,new_purchase_month_min@,new_purchase_month_std@,new_purchase_month_skew@,new_purchase_month_ptp@,new_purchase_date_ptp@,new_month_lag_min@,new_month_lag_max@,new_month_lag_ptp@,auth1_2level_month_lag_mean@,auth1_2level_month_lag_std@,auth1_2level_purchase_amount_count_mean@,auth1_2level_purchase_amount_count_std@,auth1_2level_purchase_amount_sum_mean@,auth1_2level_purchase_amount_sum_std@,auth1_2level_purchase_amount_mean_mean@,auth1_2level_purchase_amount_mean_std@,auth1_2level_purchase_amount_min_mean@,auth1_2level_purchase_amount_min_std@,auth1_2level_purchase_amount_max_mean@,auth1_2level_purchase_amount_max_std@,auth1_2level_purchase_amount_std_mean@,auth1_2level_purchase_amount_std_std@,auth1_2level_purchase_amount_ptp_mean@,auth1_2level_purchase_amount_ptp_std@,auth1_2level_installments_count_mean@,auth1_2level_installments_count_std@,auth1_2level_installments_sum_mean@,auth1_2level_installments_sum_std@,auth1_2level_installments_mean_mean@,auth1_2level_installments_mean_std@,auth1_2level_installments_min_mean@,auth1_2level_installments_min_std@,auth1_2level_installments_max_mean@,auth1_2level_installments_max_std@,auth1_2level_installments_std_mean@,auth1_2level_installments_std_std@,auth1_2level_installments_ptp_mean@,auth1_2level_installments_ptp_std@,auth0_2level_month_lag_mean@,auth0_2level_month_lag_std@,auth0_2level_purchase_amount_count_mean@,auth0_2level_purchase_amount_count_std@,auth0_2level_purchase_amount_sum_mean@,auth0_2level_purchase_amount_su

,target,feature_1@,feature_2@,feature_3@,elapsed_days@,auth1_transactions_cnt@,auth1_category_1_sum@,auth1_category_1_mean@,auth1_category_2_1.0_sum@,auth1_category_2_1.0_mean@,auth1_category_2_2.0_sum@,auth1_category_2_2.0_mean@,auth1_category_2_3.0_sum@,auth1_category_2_3.0_mean@,auth1_category_2_4.0_sum@,auth1_category_2_4.0_mean@,auth1_category_2_5.0_sum@,auth1_category_2_5.0_mean@,auth1_category_3_A_sum@,auth1_category_3_A_mean@,auth1_category_3_B_sum@,auth1_category_3_B_mean@,auth1_category_3_C_sum@,auth1_category_3_C_mean@,auth1_merchant_id_nunique@,auth1_state_id_nunique@,auth1_city_id_nunique@,auth1_subsector_id_nunique@,auth1_purchase_amount_sum@,auth1_purchase_amount_mean@,auth1_purchase_amount_max@,auth1_purchase_amount_min@,auth1_purchase_amount_std@,auth1_purchase_amount_skew@,auth1_purchase_amount_ptp@,auth1_installments_sum@,auth1_installments_mean@,auth1_installments_max@,auth1_installments_min@,auth1_installments_std@,auth1_installments_skew@,auth1_installments_ptp@,auth1_purchase_month_mean@,auth1_purchase_month_max@,auth1_purchase_month_min@,auth1_purchase_month_std@,auth1_purchase_month_skew@,auth1_purchase_month_ptp@,auth1_purchase_date_ptp@,auth1_month_lag_min@,auth1_month_lag_max@,auth1_month_lag_ptp@,auth0_transactions_cnt@,auth0_category_1_sum@,auth0_category_1_mean@,auth0_category_2_1.0_sum@,auth0_category_2_1.0_mean@,auth0_category_2_2.0_sum@,auth0_category_2_2.0_mean@,auth0_category_2_3.0_sum@,auth0_category_2_3.0_mean@,auth0_category_2_4.0_sum@,auth0_category_2_4.0_mean@,auth0_category_2_5.0_sum@,auth0_category_2_5.0_mean@,auth0_category_3_A_sum@,auth0_category_3_A_mean@,auth0_category_3_B_sum@,auth0_category_3_B_mean@,auth0_category_3_C_sum@,auth0_category_3_C_mean@,auth0_merchant_id_nunique@,auth0_state_id_nunique@,auth0_city_id_nunique@,auth0_subsector_id_nunique@,auth0_purchase_amount_sum@,auth0_purchase_amount_mean@,auth0_purchase_amount_max@,auth0_purchase_amount_min@,auth0_purchase_amount_std@,auth0_purchase_amount_skew@,auth0_purchase_amount_ptp@,auth0_installments_sum@,auth0_installments_mean@,auth0_installments_max@,auth0_installments_min@,auth0_installments_std@,auth0_installments_skew@,auth0_installments_ptp@,auth0_purchase_month_mean@,auth0_purchase_month_max@,auth0_purchase_month_min@,auth0_purchase_month_std@,auth0_purchase_month_skew@,auth0_purchase_month_ptp@,auth0_purchase_date_ptp@,auth0_month_lag_min@,auth0_month_lag_max@,auth0_month_lag_ptp@,new_transactions_cnt@,...,new_purchase_month_mean@,new_purchase_month_max@,new_purchase_month_min@,new_purchase_month_std@,new_purchase_month_skew@,new_purchase_month_ptp@,new_purchase_date_ptp@,new_month_lag_min@,new_month_lag_max@,new_month_lag_ptp@,auth1_2level_month_lag_mean@,auth1_2level_month_lag_std@,auth1_2level_purchase_amount_count_mean@,auth1_2level_purchase_amount_count_std@,auth1_2level_purchase_amount_sum_mean@,auth1_2level_purchase_amount_sum_std@,auth1_2level_purchase_amount_mean_mean@,auth1_2level_purchase_amount_mean_std@,auth1_2level_purchase_amount_min_mean@,auth1_2level_purchase_amount_min_std@,auth1_2level_purchase_amount_max_mean@,auth1_2level_purchase_amount_max_std@,auth1_2level_purchase_amount_std_mean@,auth1_2level_purchase_amount_std_std@,auth1_2level_purchase_amount_ptp_mean@,auth1_2level_purchase_amount_ptp_std@,auth1_2level_installments_count_mean@,auth1_2level_installments_count_std@,auth1_2level_installments_sum_mean@,auth1_2level_installments_sum_std@,auth1_2level_installments_mean_mean@,auth1_2level_installments_mean_std@,auth1_2level_installments_min_mean@,auth1_2level_installments_min_std@,auth1_2level_installments_max_mean@,auth1_2level_installments_max_std@,auth1_2level_installments_std_mean@,auth1_2level_installments_std_std@,auth1_2level_installments_ptp_mean@,auth1_2level_installments_ptp_std@,auth0_2level_month_lag_mean@,auth0_2level_month_lag_std@,auth0_2level_purchase_amount_count_mean@,auth0_2level_purchase_amount_count_std@,auth0_2level_purchase_amount_sum_mean@,auth0_2level_purchase_amount_su

,target,feature_1@,feature_2@,feature_3@,elapsed_days@,auth1_transactions_cnt@,auth1_category_1_sum@,auth1_category_1_mean@,auth1_category_2_1.0_sum@,auth1_category_2_1.0_mean@,auth1_category_2_2.0_sum@,auth1_category_2_2.0_mean@,auth1_category_2_3.0_sum@,auth1_category_2_3.0_mean@,auth1_category_2_4.0_sum@,auth1_category_2_4.0_mean@,auth1_category_2_5.0_sum@,auth1_category_2_5.0_mean@,auth1_category_3_A_sum@,auth1_category_3_A_mean@,auth1_category_3_B_sum@,auth1_category_3_B_mean@,auth1_category_3_C_sum@,auth1_category_3_C_mean@,auth1_merchant_id_nunique@,auth1_state_id_nunique@,auth1_city_id_nunique@,auth1_subsector_id_nunique@,auth1_purchase_amount_sum@,auth1_purchase_amount_mean@,auth1_purchase_amount_max@,auth1_purchase_amount_min@,auth1_purchase_amount_std@,auth1_purchase_amount_skew@,auth1_purchase_amount_ptp@,auth1_installments_sum@,auth1_installments_mean@,auth1_installments_max@,auth1_installments_min@,auth1_installments_std@,auth1_installments_skew@,auth1_installments_ptp@,auth1_purchase_month_mean@,auth1_purchase_month_max@,auth1_purchase_month_min@,auth1_purchase_month_std@,auth1_purchase_month_skew@,auth1_purchase_month_ptp@,auth1_purchase_date_ptp@,auth1_month_lag_min@,auth1_month_lag_max@,auth1_month_lag_ptp@,auth0_transactions_cnt@,auth0_category_1_sum@,auth0_category_1_mean@,auth0_category_2_1.0_sum@,auth0_category_2_1.0_mean@,auth0_category_2_2.0_sum@,auth0_category_2_2.0_mean@,auth0_category_2_3.0_sum@,auth0_category_2_3.0_mean@,auth0_category_2_4.0_sum@,auth0_category_2_4.0_mean@,auth0_category_2_5.0_sum@,auth0_category_2_5.0_mean@,auth0_category_3_A_sum@,auth0_category_3_A_mean@,auth0_category_3_B_sum@,auth0_category_3_B_mean@,auth0_category_3_C_sum@,auth0_category_3_C_mean@,auth0_merchant_id_nunique@,auth0_state_id_nunique@,auth0_city_id_nunique@,auth0_subsector_id_nunique@,auth0_purchase_amount_sum@,auth0_purchase_amount_mean@,auth0_purchase_amount_max@,auth0_purchase_amount_min@,auth0_purchase_amount_std@,auth0_purchase_amount_skew@,auth0_purchase_amount_ptp@,auth0_installments_sum@,auth0_installments_mean@,auth0_installments_max@,auth0_installments_min@,auth0_installments_std@,auth0_installments_skew@,auth0_installments_ptp@,auth0_purchase_month_mean@,auth0_purchase_month_max@,auth0_purchase_month_min@,auth0_purchase_month_std@,auth0_purchase_month_skew@,auth0_purchase_month_ptp@,auth0_purchase_date_ptp@,auth0_month_lag_min@,auth0_month_lag_max@,auth0_month_lag_ptp@,new_transactions_cnt@,...,new_purchase_month_mean@,new_purchase_month_max@,new_purchase_month_min@,new_purchase_month_std@,new_purchase_month_skew@,new_purchase_month_ptp@,new_purchase_date_ptp@,new_month_lag_min@,new_month_lag_max@,new_month_lag_ptp@,auth1_2level_month_lag_mean@,auth1_2level_month_lag_std@,auth1_2level_purchase_amount_count_mean@,auth1_2level_purchase_amount_count_std@,auth1_2level_purchase_amount_sum_mean@,auth1_2level_purchase_amount_sum_std@,auth1_2level_purchase_amount_mean_mean@,auth1_2level_purchase_amount_mean_std@,auth1_2level_purchase_amount_min_mean@,auth1_2level_purchase_amount_min_std@,auth1_2level_purchase_amount_max_mean@,auth1_2level_purchase_amount_max_std@,auth1_2level_purchase_amount_std_mean@,auth1_2level_purchase_amount_std_std@,auth1_2level_purchase_amount_ptp_mean@,auth1_2level_purchase_amount_ptp_std@,auth1_2level_installments_count_mean@,auth1_2level_installments_count_std@,auth1_2level_installments_sum_mean@,auth1_2level_installments_sum_std@,auth1_2level_installments_mean_mean@,auth1_2level_installments_mean_std@,auth1_2level_installments_min_mean@,auth1_2level_installments_min_std@,auth1_2level_installments_max_mean@,auth1_2level_installments_max_std@,auth1_2level_installments_std_mean@,auth1_2level_installments_std_std@,auth1_2level_installments_ptp_mean@,auth1_2level_installments_ptp_std@,auth0_2level_month_lag_mean@,auth0_2level_month_lag_std@,auth0_2level_purchase_amount_count_mean@,auth0_2level_purchase_amount_count_std@,auth0_2level_purchase_amount_sum_mean@,auth0_2level_purchase_amount_su

In [124]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
warnings.simplefilter(action='ignore', category=FutureWarning)


param = {'num_leaves': 100,
         'min_data_in_leaf': 30, 
         'objective':'regression',
         'max_depth': 6,
         'learning_rate': 0.005,
         "min_child_samples": 20,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 0.1,
         "verbosity": -1}

folds = KFold(n_splits=5, shuffle=True, random_state=15)
oof = np.zeros(len(train))
predictions = np.zeros(len(test))
start = time.time()
feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train.values, y.values)):
    print("fold n°{}".format(fold_))
    trn_data = lgb.Dataset(train.iloc[trn_idx][features], label=y.iloc[trn_idx], categorical_feature=categorical_feats)
    val_data = lgb.Dataset(train.iloc[val_idx][features], label=y.iloc[val_idx], categorical_feature=categorical_feats)

    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 200)
    oof[val_idx] = clf.predict(train.iloc[val_idx][features], num_iteration=clf.best_iteration)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = features
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    predictions += clf.predict(test[features], num_iteration=clf.best_iteration) / folds.n_splits

print("CV score: {:<8.5f}".format(mean_squared_error(oof, y)**0.5))

fold n°0
Training until validation scores don't improve for 200 rounds.
[100]	training's rmse: 3.74683	valid_1's rmse: 3.8068
[200]	training's rmse: 3.6891	valid_1's rmse: 3.76936
[300]	training's rmse: 3.64993	valid_1's rmse: 3.74667
[400]	training's rmse: 3.62076	valid_1's rmse: 3.73017
[500]	training's rmse: 3.59836	valid_1's rmse: 3.71981
[600]	training's rmse: 3.58143	valid_1's rmse: 3.71272
[700]	training's rmse: 3.56812	valid_1's rmse: 3.70846
[800]	training's rmse: 3.55613	valid_1's rmse: 3.70526
[900]	training's rmse: 3.54615	valid_1's rmse: 3.7029
[1000]	training's rmse: 3.53647	valid_1's rmse: 3.70107
[1100]	training's rmse: 3.52765	valid_1's rmse: 3.69957
[1200]	training's rmse: 3.51911	valid_1's rmse: 3.69881
[1300]	training's rmse: 3.51139	valid_1's rmse: 3.69823
[1400]	training's rmse: 3.50361	valid_1's rmse: 3.69764
[1500]	training's rmse: 3.49613	valid_1's rmse: 3.69731
[1600]	training's rmse: 3.48897	valid_1's rmse: 3.69709
[1700]	training's rmse: 3.48189	valid_1's rm

In [128]:
for col in train.columns:
    if col.count('@'):
        utils.to_pkl_gzip(obj=train[col].values, path=f'../features/4_winner/{feat_no}train_{col}')
        utils.to_pkl_gzip(obj=test[col].values, path=f'../features/4_winner/{feat_no}test_{col}')
test.head()

,target,feature_1@,feature_2@,feature_3@,elapsed_days@,auth1_transactions_cnt@,auth1_category_1_sum@,auth1_category_1_mean@,auth1_category_2_1.0_sum@,auth1_category_2_1.0_mean@,auth1_category_2_2.0_sum@,auth1_category_2_2.0_mean@,auth1_category_2_3.0_sum@,auth1_category_2_3.0_mean@,auth1_category_2_4.0_sum@,auth1_category_2_4.0_mean@,auth1_category_2_5.0_sum@,auth1_category_2_5.0_mean@,auth1_category_3_A_sum@,auth1_category_3_A_mean@,auth1_category_3_B_sum@,auth1_category_3_B_mean@,auth1_category_3_C_sum@,auth1_category_3_C_mean@,auth1_merchant_id_nunique@,auth1_state_id_nunique@,auth1_city_id_nunique@,auth1_subsector_id_nunique@,auth1_purchase_amount_sum@,auth1_purchase_amount_mean@,auth1_purchase_amount_max@,auth1_purchase_amount_min@,auth1_purchase_amount_std@,auth1_purchase_amount_skew@,auth1_purchase_amount_ptp@,auth1_installments_sum@,auth1_installments_mean@,auth1_installments_max@,auth1_installments_min@,auth1_installments_std@,auth1_installments_skew@,auth1_installments_ptp@,auth1_purchase_month_mean@,auth1_purchase_month_max@,auth1_purchase_month_min@,auth1_purchase_month_std@,auth1_purchase_month_skew@,auth1_purchase_month_ptp@,auth1_purchase_date_ptp@,auth1_month_lag_min@,auth1_month_lag_max@,auth1_month_lag_ptp@,auth0_transactions_cnt@,auth0_category_1_sum@,auth0_category_1_mean@,auth0_category_2_1.0_sum@,auth0_category_2_1.0_mean@,auth0_category_2_2.0_sum@,auth0_category_2_2.0_mean@,auth0_category_2_3.0_sum@,auth0_category_2_3.0_mean@,auth0_category_2_4.0_sum@,auth0_category_2_4.0_mean@,auth0_category_2_5.0_sum@,auth0_category_2_5.0_mean@,auth0_category_3_A_sum@,auth0_category_3_A_mean@,auth0_category_3_B_sum@,auth0_category_3_B_mean@,auth0_category_3_C_sum@,auth0_category_3_C_mean@,auth0_merchant_id_nunique@,auth0_state_id_nunique@,auth0_city_id_nunique@,auth0_subsector_id_nunique@,auth0_purchase_amount_sum@,auth0_purchase_amount_mean@,auth0_purchase_amount_max@,auth0_purchase_amount_min@,auth0_purchase_amount_std@,auth0_purchase_amount_skew@,auth0_purchase_amount_ptp@,auth0_installments_sum@,auth0_installments_mean@,auth0_installments_max@,auth0_installments_min@,auth0_installments_std@,auth0_installments_skew@,auth0_installments_ptp@,auth0_purchase_month_mean@,auth0_purchase_month_max@,auth0_purchase_month_min@,auth0_purchase_month_std@,auth0_purchase_month_skew@,auth0_purchase_month_ptp@,auth0_purchase_date_ptp@,auth0_month_lag_min@,auth0_month_lag_max@,auth0_month_lag_ptp@,new_transactions_cnt@,...,new_purchase_month_mean@,new_purchase_month_max@,new_purchase_month_min@,new_purchase_month_std@,new_purchase_month_skew@,new_purchase_month_ptp@,new_purchase_date_ptp@,new_month_lag_min@,new_month_lag_max@,new_month_lag_ptp@,auth1_2level_month_lag_mean@,auth1_2level_month_lag_std@,auth1_2level_purchase_amount_count_mean@,auth1_2level_purchase_amount_count_std@,auth1_2level_purchase_amount_sum_mean@,auth1_2level_purchase_amount_sum_std@,auth1_2level_purchase_amount_mean_mean@,auth1_2level_purchase_amount_mean_std@,auth1_2level_purchase_amount_min_mean@,auth1_2level_purchase_amount_min_std@,auth1_2level_purchase_amount_max_mean@,auth1_2level_purchase_amount_max_std@,auth1_2level_purchase_amount_std_mean@,auth1_2level_purchase_amount_std_std@,auth1_2level_purchase_amount_ptp_mean@,auth1_2level_purchase_amount_ptp_std@,auth1_2level_installments_count_mean@,auth1_2level_installments_count_std@,auth1_2level_installments_sum_mean@,auth1_2level_installments_sum_std@,auth1_2level_installments_mean_mean@,auth1_2level_installments_mean_std@,auth1_2level_installments_min_mean@,auth1_2level_installments_min_std@,auth1_2level_installments_max_mean@,auth1_2level_installments_max_std@,auth1_2level_installments_std_mean@,auth1_2level_installments_std_std@,auth1_2level_installments_ptp_mean@,auth1_2level_installments_ptp_std@,auth0_2level_month_lag_mean@,auth0_2level_month_lag_std@,auth0_2level_purchase_amount_count_mean@,auth0_2level_purchase_amount_count_std@,auth0_2level_purchase_amount_sum_mean@,auth0_2level_purchase_amount_su

In [129]:
submit = pd.read_csv('../input/sample_submission.csv')
submit[target] = predictions
submit.head(40)

,card_id,target
0,C_ID_0ab67a22ab,-2.861534
1,C_ID_130fd0cbdd,-0.478100
2,C_ID_b709037bc5,-1.232105
3,C_ID_d27d835a9f,-0.135344
4,C_ID_2b5e3df5c2,-1.095355
5,C_ID_5814b4f13c,0.176654
6,C_ID_a1b3c75277,0.166560
7,C_ID_f7cada36d3,0.411831
8,C_ID_9d2bc8dfc4,-0.811134
9,C_ID_6d8dba8475,-0.974287
